## First stab at visualizing ATL03


In [41]:
import pandas as pd
import geopandas as gpd
import h5py
import glob
import os
import matplotlib.pyplot as plt
# from bokeh.plotting import figure, output_file, show
# from bokeh.models import ColumnDataSource, LinearColorMapper
import numpy as np
from ipyleaflet import Map, basemaps, basemap_to_tiles, GeoData, LayersControl
import geopandas
import json
from readers.read_HDF5_ATL03 import read_HDF5_ATL03
from readers.get_ATL03_x_atc import get_ATL03_x_atc

In [42]:
import getpass

def list_files_local(path):
    """ Get file list form local folder. """
    from glob import glob
    return glob(path)


In [43]:
prodname = 'ATL03'
site = 'seeps-small-helheim'
path2files = str('/home/jovyan/' + prodname + '/' + site + '/')
# print(path2files)
hdf5_flist = glob.glob(str(path2files +'*.h5'))
# print(hdf5_flist)
f = h5py.File(hdf5_flist[0], 'r')
print(f)

<HDF5 file "processed_ATL03_20181227141518_13740103_001_01_gt2r.h5" (mode r)>


In [44]:
import pyproj
from astropy.time import Time

def gps2dyr(time):
    """ Converte GPS time to decimal years. """
    return Time(time, format='gps').decimalyear

def track_type(time, lat, tmax=1):
    """
    Separate tracks into ascending and descending.
    
    Defines tracks as segments with time breaks > tmax,
    and tests whether lat increases or decreases w/time.
    """
    tracks = np.zeros(lat.shape)  # generate track segment
    tracks[0:np.argmax(np.abs(lat))] = 1  # set values for segment
    i_asc = np.zeros(tracks.shape, dtype=bool)  # output index array

    # Loop trough individual secments
    for track in np.unique(tracks):
    
        i_track, = np.where(track == tracks)  # get all pts from seg
    
        if len(i_track) < 2: continue
    
        # Test if lat increases (asc) or decreases (des) w/time
        i_min = time[i_track].argmin()
        i_max = time[i_track].argmax()
        lat_diff = lat[i_track][i_max] - lat[i_track][i_min]
    
        # Determine track type
        if lat_diff > 0:  i_asc[i_track] = True
    
    return i_asc, np.invert(i_asc)  # index vectors


def transform_coord(proj1, proj2, x, y):
    """
    Transform coordinates from proj1 to proj2 (EPSG num).

    Example EPSG projs:
        Geodetic (lon/lat): 4326
        Polar Stereo AnIS (x/y): 3031
        Polar Stereo GrIS (x/y): 3413
    """
    # Set full EPSG projection strings
    proj1 = pyproj.Proj("+init=EPSG:"+str(proj1))
    proj2 = pyproj.Proj("+init=EPSG:"+str(proj2))
    return pyproj.transform(proj1, proj2, x, y)  # convert


In [45]:
import matplotlib.pyplot as plt
%matplotlib widget

def read_h5(fname, vnames=[]):
    """ Simple HDF5 reader. """
    with h5py.File(fname, 'r') as f:
        return [f[v][:] for v in vnames]

In [46]:
# files = list_files_local(path2files)
# print(files)
fname = hdf5_flist[1]
print(fname)

#!h5ls -r /home/jovyan/ATL06/seeps/processed_ATL06_20181030170315_04900103_001_01.h5
# !h5ls -r /home/jovyan/ATL03/seeps/processed_ATL03_20190202123455_05510203_001_01.h5


/home/jovyan/ATL03/seeps-small-helheim/processed_ATL03_20190212003725_06960205_001_01_gt2r.h5


In [47]:
import h5py
import numpy as np

def read_atl03(fname, bbox=None):
    print(fname)
    """ 
    Read 1 ATL06 file and output 6 reduced files. 
    
    Extract variables of interest and separate the ATL06 file 
    into each beam (ground track) and ascending/descending orbits.
    """
    f = h5py.File(fname,'r')
    print(f.keys())
    # Each beam is a group
    group = ['/gt1l', '/gt1r', '/gt2l', '/gt2r', '/gt3l', '/gt3r']
    # Loop trough beams
    for k,g in enumerate(group):
    
        #-----------------------------------#
        # 1) Read in data for a single beam #
        #-----------------------------------#
    
        # Load variables into memory (more can be added!)
        with h5py.File(fname, 'r') as fi:
            print(fi[g].keys())
            if g+'/heights' in fi.keys():
                
                print('Found some data on beam' + g)
                print(fi[g+'/heights'].keys()) 
                
                lat = fi[g+'/heights/lat_ph'][:]
                lon = fi[g+'/heights/lon_ph'][:]
                h = fi[g+'/heights/h_ph'][:]
                conf = fi[g+'/heights/signal_conf_ph']

                land_ice_class = conf[:,3]
          
        #-----------------------------------#
        # 3) Filter data #
        #-----------------------------------#
                mask = (land_ice_class == 4) & (np.abs(h) < 10e3)
                lat,lon,h = lat[mask],lon[mask],h[mask]
        
        #-----------------------#
        # 4) Save selected data #
        #-----------------------#
        
        # Define output file name
                ofile = fname.replace('.h5', '_'+g[1:]+'.h5')
                
        # Save variables
                with h5py.File(ofile, 'w') as f:
                    f['lon'] = lon
                    f['lat'] = lat
                    f['h_elv'] = h
            
                    print('out ->', ofile)
                

In [50]:
from readers.read_HDF5_ATL03 import read_HDF5_ATL03
from readers.get_ATL03_x_atc import get_ATL03_x_atc

# read the IS2 data with Tyler's ATL03 reader:
IS2_atl03_mds, IS2_atl03_attrs, IS2_atl03_beams =read_HDF5_ATL03('/home/jovyan/ATL03/seeps-small-helheim/processed_ATL03_20181227141518_13740103_001_01.h5')

# add x_atc to the ATL03 data structure (this function adds to the LS2_ATL03_mds dictionary)
get_ATL03_x_atc(IS2_atl03_mds, IS2_atl03_attrs, IS2_atl03_beams)

print(IS2_atl03_mds)

#-- select the 1r beam from ATL03
D3 = IS2_atl03_mds['gt1r']

#-- create scatter plot of photon data (e.g., photon elevation vs x_atc)
%matplotlib widget
f1,ax = plt.subplots(num=1,figsize=(10,6))
plt.plot(D3['heights']['x_atc'], D3['heights']['h_ph'],'*g',markersize=0.1)
ax.set_xlabel('x_atc, m')
ax.set_ylabel('h, m')
plt.show()

FigureCanvasNbAgg()

In [92]:
print(IS2_atl03_mds)


#-- create scatter plot of photon data (e.g., photon elevation vs x_atc)
DX = IS2_atl03_mds  # all 6 lasers

%matplotlib widget
f1,ax = plt.subplots(num=1,figsize=(10,6))

xra = DX['gt1r']['heights']['x_atc']
plt.plot((xra-xra[0])/1000, DX['gt1r']['heights']['h_ph'],'*g',markersize=0.1)

xla = DX['gt1l']['heights']['x_atc']
plt.plot((xla-xla[0])/1000, DX['gt1l']['heights']['h_ph'],'*b',markersize=0.1)

# xrb = DX['gt2r']['heights']['x_atc']
# plt.plot((xrb-xrb[0])/1000, DX['gt2r']['heights']['h_ph'],'*k',markersize=0.1)

# xlb = DX['gt2l']['heights']['x_atc']
# plt.plot((xlb-xlb[0])/1000, DX['gt2l']['heights']['h_ph'],'*r',markersize=0.1)

ax.set_xlabel('x_atc (km)')
ax.set_ylabel('h, m')
plt.show()

{'gt1l': {'heights': {'delta_time': array([31155433.5907155 , 31155433.5907155 , 31155433.5908155 , ...,
       31155435.51591555, 31155435.51601555, 31155435.51601555]), 'dist_ph_across': array([1895.9623, 1895.9618, 1895.3619, ..., 1898.4171, 1898.4187,
       1898.5637], dtype=float32), 'dist_ph_along': array([ 0.32990095,  0.32990086,  1.0980086 , ..., 19.192476  ,
       19.900236  , 19.88591   ], dtype=float32), 'h_ph': array([1628.8528, 1628.9874, 1787.3087, ..., 1837.7   , 1837.478 ,
       1799.166 ], dtype=float32), 'lat_ph': array([66.73370481, 66.73370481, 66.73371227, ..., 66.85548012,
       66.85548643, 66.85548615]), 'lon_ph': array([-38.51077335, -38.51077334, -38.51076182, ..., -38.54646078,
       -38.54646267, -38.54646591]), 'pce_mframe_cnt': array([410016117, 410016117, 410016117, ..., 410016213, 410016213,
       410016213], dtype=uint32), 'ph_id_channel': array([119, 118,  60, ...,  60,  60,  57], dtype=uint8), 'ph_id_count': array([1, 1, 1, ..., 2, 1, 1], dtype

FigureCanvasNbAgg()

In [87]:
# print(IS2_atl03_mds)
x1rlat = DX['gt1r']['heights']['lat_ph']
x1rlon = DX['gt1r']['heights']['lon_ph']
x1llat = DX['gt1l']['heights']['lat_ph']
x1llon = DX['gt1l']['heights']['lon_ph']

x2rlat = DX['gt2r']['heights']['lat_ph']
x2rlon = DX['gt2r']['heights']['lon_ph']
x2llat = DX['gt2l']['heights']['lat_ph']
x2llon = DX['gt2l']['heights']['lon_ph']

x3rlat = DX['gt3r']['heights']['lat_ph']
x3rlon = DX['gt3r']['heights']['lon_ph']
x3llat = DX['gt3l']['heights']['lat_ph']
x3llon = DX['gt3l']['heights']['lon_ph']


In [91]:
import csv
 
myData = [np.transpose(x1rlat[::1000]), np.transpose(x1rlon[::1000])]
print(len(np.transpose(myData)))
myFile = open('seep-lat-lon-x1r-1000.csv', 'w')
with myFile:
    writer = csv.writer(myFile)
    writer.writerows(np.transpose(myData)) 
print("Writing x1r complete")

myData = [np.transpose(x1llat[::1000]), np.transpose(x1llon[::1000])]
print(len(np.transpose(myData))) 
myFile = open('seep-lat-lon-x1l-1000.csv', 'w')
with myFile:
    writer = csv.writer(myFile)
    writer.writerows(np.transpose(myData))
print("Writing x1l complete")


myData = [np.transpose(x2rlat[::1000]), np.transpose(x2rlon[::1000])]
print(len(np.transpose(myData))) 
myFile = open('seep-lat-lon-x2r-1000.csv', 'w')
with myFile:
    writer = csv.writer(myFile)
    writer.writerows(np.transpose(myData))
print("Writing x2r complete")

myData = [np.transpose(x2llat[::1000]), np.transpose(x2llon[::1000])]
print(len(np.transpose(myData))) 
myFile = open('seep-lat-lon-x2l-1000.csv', 'w')
with myFile:
    writer = csv.writer(myFile)
    writer.writerows(np.transpose(myData))
print("Writing x2l complete")


myData = [np.transpose(x3rlat[::1000]), np.transpose(x3rlon[::1000])]
print(len(np.transpose(myData))) 
myFile = open('seep-lat-lon-x3r-1000.csv', 'w')
with myFile:
    writer = csv.writer(myFile)
    writer.writerows(np.transpose(myData))
print("Writing x3r complete")

myData = [np.transpose(x3llat[::1000]), np.transpose(x3llon[::1000])]
print(len(np.transpose(myData))) 
myFile = open('seep-lat-lon-x3l-1000.csv', 'w')
with myFile:
    writer = csv.writer(myFile)
    writer.writerows(np.transpose(myData))
print("Writing x3l complete")

203
Writing x1r complete
52
Writing x1l complete
196
Writing x2r complete
65
Writing x2l complete
219
Writing x3r complete
61
Writing x3l complete
